In [1]:
!pip install SPARQLWrapper

     |████████████████████████████████| 235kB 6.2MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 


In [2]:
import pandas as pd
import ast
import math
from textblob import TextBlob
import re
import numpy as np
import csv
# import seaborn
import statistics
from SPARQLWrapper import SPARQLWrapper, JSON
import json

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [4]:
root_dir = '/content/drive/MyDrive/ProgettoAI/dataset/'
clinton = pd.read_csv(root_dir+'clinton_corrected.csv')



In [5]:
class DbpediaObject:

  def __init__(self, string):
    json_parsed = ast.literal_eval(string)
    self.confidence = float(json_parsed['@confidence'])
    self.text = json_parsed['@text']
    self.entities = []
    try:
      self._set_entities(json_parsed['Resources'])
    except:
      None



  def _set_entities(self, lista):
    for el in lista:
      if(el['@surfaceForm'] != 't.co'):
        self.entities.append(Entity(el))


In [6]:
class Entity:
  def __init__(self, json_parsed):
    self.original_text = json_parsed['@surfaceForm']
    self.label = json_parsed['@URI'].split('/')[-1]
    self.uri = json_parsed['@URI']
    self.similarity = json_parsed['@similarityScore']
    self.types = []
    self.schema = []
    self._set_types(json_parsed['@types'])

  def _set_types(self, raw_list):
      raw_split = raw_list.split(',')
      for el in raw_split:
        x = el.split(':')
        if x[0] == 'DBpedia':
          self.types.append(x[1])
        if x[0] == 'Wikidata':
          self.wikidata = x[1]
        if x[0] == 'Schema':
          self.schema.append(x[1])




In [7]:
def count_empty_list(lista):
  count = 0
  for el in lista:
    if len(el) == 0:
      count = count +1
  return count

In [8]:
def entita_filter_by_type(tipo, df_entita):
  cit = list()
  count_cit = list()
  entities_cit = list()
  plain_text = list()

  for index, row in df_entita.iterrows():
    if tipo in row['entita'].types:
      cit.append(row['entita'].label)
      count_cit.append(row['count'])
      entities_cit.append(row['entita'].uri)
      plain_text.append(row['plain_text'])
  return pd.DataFrame(list(zip(cit,count_cit,entities_cit,plain_text)), columns=['label', 'count', 'uri','plain_text']).sort_values('count', ascending=False)


#clinton

In [9]:
lista = []
for index, el in clinton.iterrows():
  try:
    lista.append(DbpediaObject(el['dbpedia']))
  except:
    print(index)
clinton['dbpedia_object'] = lista

In [10]:
clinton=clinton.drop('Unnamed: 0', axis=1)
clinton=clinton.drop('Unnamed: 0.1', axis=1)
clinton

,handle,text,is_retweet,original_author,time,entities,extended_entities,dbpedia,dbpedia_object
0,HillaryClinton,The question in this election: Who can put the...,False,NaN,2016-09-28T00:22:34,{'media': [{'display_url': 'pic.twitter.com/Xr...,{'media': [{'display_url': 'pic.twitter.com/Xr...,{'@text': 'The question in this election: Who ...,<__main__.DbpediaObject object at 0x7f0b894b7048>
1,HillaryClinton,"Last night, Donald Trump said not paying taxes...",True,timkaine,2016-09-27T23:45:00,{'media': [{'display_url': 'pic.twitter.com/t0...,{'media': [{'display_url': 'pic.twitter.com/t0...,"{'@text': 'Last night, Donald Trump said not p...",<__main__.DbpediaObject object at 0x7f0b89508fd0>
2,HillaryClinton,Couldn't be more proud of @HillaryClinton. Her...,True,POTUS,2016-09-27T23:26:40,"{'user_mentions': [{'id_str': '1536791610', 'n...",NaN,"{'@text': ""Couldn't be more proud of @HillaryC...",<__main__.DbpediaObject object at 0x7f0b894b7278>
3,HillaryClinton,"If we stand together, there's nothing we can't...",False,NaN,2016-09-27T23:08:41,{'media': [{'display_url': 'pic.twitter.com/Q3...,{'media': [{'display_url': 'pic.twitter.com/Q3...,"{'@text': ""If we stand together, there's nothi...",<__main__.DbpediaObject object at 0x7f0b894b72e8>
4,HillaryClinton,Both candidates were asked about how they'd co...,False,NaN,2016-09-27T22:30:27,"{'user_mentions': [], 'symbols': [], 'urls': [...",NaN,"{'@text': ""Both candidates were asked about ho...",<__main__.DbpediaObject object at 0x7f0b894b7198>
...,...,...,...,...,...,...,...,...,...
3221,HillaryClinton,"A great day in East Harlem, in photos. https:/...",False,NaN,2016-04-17T18:20:35,{'media': [{'display_url': 'pic.twitter.com/jc...,{'media': [{'display_url': 'pic.twitter.com/jc...,"{'@text': 'A great day in East Harlem, in phot...",<__main__.DbpediaObject object at 0x7f0b88d9b390>
3222,HillaryClinton,Meet three of the kids covered by the Children...,True,HFA,2016-04-17T18:07:14,{'user_mentions': [{'id_str': '702147673651027...,NaN,"{'@text': ""Meet three of the kids covered by t...",<__main__.DbpediaObject object at 0x7f0b88e07eb8>
3223,HillaryClinton,"Your Sunday afternoon read: ""In a word, Clinto...",True,HillaryforPA,2016-04-17T16:39:42,"{'user_mentions': [{'id_str': '3061615740', 'n...",NaN,"{'@text': 'Your Sunday afternoon read: ""In a w...",<__main__.DbpediaObject object at 0x7f0b88d9b8d0>
3224,HillaryClinton,“Muchos neoyorquinos luchan por pagar la renta...,False,NaN,2016-04-17T16:15:32,"{'user_mentions': [], 'symbols': [], 'urls': [...",NaN,{'@text': '“Muchos neoyorquinos luchan por pag...,<__main__.DbpediaObject object at 0x7f0b88d9ba90>


In [11]:
clinton_list_entities = list(x.entities for x in clinton['dbpedia_object'])


In [12]:
x = round(count_empty_list(clinton_list_entities)/len(clinton_list_entities)*100,2)
print("% of tweet without entities: ", x)

% of tweet without entities:  12.43


In [13]:
num_entities = [i.label for x in clinton_list_entities for i in x]
print("Different entities found", len(set(num_entities)))

Different entities found 1718


In [14]:
num_entities = [i.original_text for x in clinton_list_entities for i in x]
print("Different labels found", len(set(num_entities)))

Different labels found 1901


In [15]:
num_entities = [i for x in clinton_list_entities for i in x]
print("Entities found", len(num_entities))

Entities found 6615


In [16]:
entities_count_clinton = dict((el.label,el) for el in [i for x in clinton_list_entities for i in x])
entita_clinton = list(entities_count_clinton.values())
entities_count_clinton = list(np.zeros(len(entita_clinton), dtype=int))
clinton_labels = [x.label for x in entita_clinton]
clinton_types =  [x.types for x in entita_clinton]
# clinton_schema =  [x.schema for x in entita_clinton]

for el in clinton['dbpedia_object']:
  for entity in el.entities:
    index = clinton_labels.index(entity.label)
    entities_count_clinton[index] = entities_count_clinton[index] + 1

entities_frame = pd.DataFrame(list(zip(entita_clinton, clinton_labels,entities_count_clinton,clinton_types)), columns=['entita', 'label', 'count','types'])
# entities_frame.sort_values('count', ascending=False)

In [17]:
clinton_types = list(set(tipo for el in entities_frame['types'] for tipo in el))
clinton_types.append('Without_type')
clinton_types_count = list(np.zeros(len(clinton_types),dtype=int))

for ent_list in clinton_list_entities:
  for ent in ent_list:
    if not ent.types:
      clinton_types_count[len(clinton_types_count)-1] = clinton_types_count[len(clinton_types_count)-1] + 1
    for tipo in ent.types:
      index = clinton_types.index(tipo)
      clinton_types_count[index] = clinton_types_count[index] + 1
tipi_clinton_frame = pd.DataFrame(list(zip(clinton_types, clinton_types_count)), columns = ['tipo', 'count'])
tipi_clinton_frame.sort_values('count', ascending=False).iloc[0:20]

,tipo,count
164,Without_type,3455
90,Agent,1720
44,Person,1265
51,Politician,1180
66,Location,753
142,Place,753
103,PopulatedPlace,700
12,Organisation,428
94,Country,422
107,Work,360


In [18]:
# total = []
# for en in entities_frame['entita']:
  # partial = []
  # for i in range(0,len(clinton)):
    # if en.label in [x.label for x in clinton.iloc[i]['dbpedia_object'].entities]:
      # partial.append(i)
  # total.append(partial)

In [19]:
# with open("clinton_entities_occurrence.csv", "w", newline="") as f:
    # writer = csv.writer(f)
    # writer.writerows(total)

In [20]:
total = []
with open('/content/drive/MyDrive/ProgettoAI/dataset/clinton_entities_occurrence.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
      total.append({int(x) for x in row})     

In [21]:
entities_frame['occurrences'] = total

In [22]:
plain_text = []
for index, el in entities_frame.iterrows():
  partial = set()
  for i in el['occurrences']:
    for entity in clinton.iloc[i]['dbpedia_object'].entities:
      if entity.label == el['label']:
        partial.add(entity.original_text)
  plain_text.append(partial)


In [23]:
entities_frame['plain_text'] = plain_text
entities_frame.sort_values('count', ascending=False)

,entita,label,count,types,occurrences,plain_text
1,<__main__.Entity object at 0x7f0b88df6da0>,Donald_Trump,884,"[Person, Agent, Politician]","{2048, 1, 2050, 2051, 6, 7, 2055, 9, 10, 2056,...","{Donald Trump, Trump, President Trump}"
2,<__main__.Entity object at 0x7f0b88e07f60>,President_of_the_United_States,396,[],"{2, 10, 16, 25, 2081, 2083, 2085, 2087, 2088, ...","{President, @POTUS, United States presidency, ..."
8,<__main__.Entity object at 0x7f0b88e0e710>,United_States,336,"[PopulatedPlace, Place, Location, Country]","{2049, 1026, 1027, 1542, 9, 2057, 1551, 2063, ...","{U.S., USA, Estados Unidos, americanos, US, Un..."
52,<__main__.Entity object at 0x7f0b88e34048>,Donald_Trump_on_social_media,116,[],"{2057, 2573, 2062, 1042, 2067, 2578, 2069, 258...","{@realdonaldtrump, @realDonaldTrump, @RealDona..."
44,<__main__.Entity object at 0x7f0b88d9b908>,Hillary_Clinton,108,"[Person, Agent, Politician]","{1536, 1540, 1542, 2570, 1548, 1559, 1560, 156...","{Hillary Clinton, Clinton}"
...,...,...,...,...,...,...
906,<__main__.Entity object at 0x7f0b891a5550>,Iron,1,[],{1402},{fe}
269,<__main__.Entity object at 0x7f0b893eb6a0>,Symposium,1,[],{340},{Symposium}
270,<__main__.Entity object at 0x7f0b893eb978>,Weather,1,[],{343},{weather}
903,<__main__.Entity object at 0x7f0b891aabe0>,Trust_law,1,[],{1399},{trusts}


In [ ]:
for i in tipi_clinton_frame.sort_values('count', ascending=False).iloc[1:]['tipo']:
  print('-------------------------------','TIPO:',i,'--------------------------------------')
  display(entita_filter_by_type(i,entities_frame).iloc[:5])

------------------------------- TIPO: Agent --------------------------------------


,label,count,uri,plain_text
0,Donald_Trump,884,http://dbpedia.org/resource/Donald_Trump,"{President Trump, Trump, Donald Trump}"
7,Hillary_Clinton,108,http://dbpedia.org/resource/Hillary_Clinton,"{Clinton, Hillary Clinton}"
14,Republican_Party_(United_States),46,http://dbpedia.org/resource/Republican_Party_(...,"{Republicans, Republican, Republican party, Re..."
9,Barack_Obama,40,http://dbpedia.org/resource/Barack_Obama,"{Obama, Barack Obama, President Obama}"
33,United_States_Senate,29,http://dbpedia.org/resource/United_States_Senate,"{United States Senate, Senator, Senate, senator}"


------------------------------- TIPO: Person --------------------------------------


,label,count,uri,plain_text
0,Donald_Trump,884,http://dbpedia.org/resource/Donald_Trump,"{President Trump, Trump, Donald Trump}"
3,Hillary_Clinton,108,http://dbpedia.org/resource/Hillary_Clinton,"{Clinton, Hillary Clinton}"
4,Barack_Obama,40,http://dbpedia.org/resource/Barack_Obama,"{Obama, Barack Obama, President Obama}"
29,Mike_Pence,26,http://dbpedia.org/resource/Mike_Pence,"{VP, Mike Pence}"
9,Bill_Clinton,13,http://dbpedia.org/resource/Bill_Clinton,"{Clinton, Bill Clinton}"


------------------------------- TIPO: Politician --------------------------------------


...

,label,count,uri,plain_text
0,Brian_Flynn,1,http://dbpedia.org/resource/Brian_Flynn,{Flynn}


------------------------------- TIPO: Musical --------------------------------------


,label,count,uri,plain_text
0,Roberta,1,http://dbpedia.org/resource/Roberta,{Roberta}


------------------------------- TIPO: ComicsCharacter --------------------------------------


,label,count,uri,plain_text
0,Kaine_Parker,1,http://dbpedia.org/resource/Kaine_Parker,{Kaine}


### Sentiment


In [25]:
sentiments = clinton['text'].apply(TextBlob)

In [26]:
polarities = [x.sentiment.polarity for x in sentiments]
subjectivities = [x.sentiment.subjectivity for x in sentiments]

In [27]:
clinton['sentiment'] = polarities
clinton['subjectivity'] = subjectivities
clinton

,handle,text,is_retweet,original_author,time,entities,extended_entities,dbpedia,dbpedia_object,sentiment,subjectivity
0,HillaryClinton,The question in this election: Who can put the...,False,NaN,2016-09-28T00:22:34,{'media': [{'display_url': 'pic.twitter.com/Xr...,{'media': [{'display_url': 'pic.twitter.com/Xr...,{'@text': 'The question in this election: Who ...,<__main__.DbpediaObject object at 0x7f0b894b7048>,0.300000,0.300000
1,HillaryClinton,"Last night, Donald Trump said not paying taxes...",True,timkaine,2016-09-27T23:45:00,{'media': [{'display_url': 'pic.twitter.com/t0...,{'media': [{'display_url': 'pic.twitter.com/t0...,"{'@text': 'Last night, Donald Trump said not p...",<__main__.DbpediaObject object at 0x7f0b89508fd0>,0.107143,0.354762
2,HillaryClinton,Couldn't be more proud of @HillaryClinton. Her...,True,POTUS,2016-09-27T23:26:40,"{'user_mentions': [{'id_str': '1536791610', 'n...",NaN,"{'@text': ""Couldn't be more proud of @HillaryC...",<__main__.DbpediaObject object at 0x7f0b894b7278>,0.300000,0.413333
3,HillaryClinton,"If we stand together, there's nothing we can't...",False,NaN,2016-09-27T23:08:41,{'media': [{'display_url': 'pic.twitter.com/Q3...,{'media': [{'display_url': 'pic.twitter.com/Q3...,"{'@text': ""If we stand together, there's nothi...",<__main__.DbpediaObject object at 0x7f0b894b72e8>,0.350000,0.694444
4,HillaryClinton,Both candidates were asked about how they'd co...,False,NaN,2016-09-27T22:30:27,"{'user_mentions': [], 'symbols': [], 'urls': [...",NaN,"{'@text': ""Both candidates were asked about ho...",<__main__.DbpediaObject object at 0x7f0b894b7198>,0.100000,0.650000
...,...,...,...,...,...,...,...,...,...,...,...
3221,HillaryClinton,"A great day in East Harlem, in photos. https:/...",False,NaN,2016-04-17T18:20:35,{'media': [{'display_url': 'pic.twitter.com/jc...,{'media': [{'display_url': 'pic.twitter.com/jc...,"{'@text': 'A great day in East Harlem, in phot...",<__main__.DbpediaObject object at 0x7f0b88d9b390>,0.800000,0.750000
3222,HillaryClinton,Meet three of the kids covered by the Children...,True,HFA,2016-04-17T18:07:14,{'user_mentions': [{'id_str': '702147673651027...,NaN,"{'@text': ""Meet three of the kids covered by t...",<__main__.DbpediaObject object at 0x7f0b88e07eb8>,0.000000,0.000000
3223,HillaryClinton,"Your Sunday afternoon read: ""In a word, Clinto...",True,HillaryforPA,2016-04-17T16:39:42,"{'user_mentions': [{'id_str': '3061615740', 'n...",NaN,"{'@text': 'Your Sunday afternoon read: ""In a w...",<__main__.DbpediaObject object at 0x7f0b88d9b8d0>,0.000000,0.000000
3224,HillaryClinton,“Muchos neoyorquinos luchan por pagar la renta...,False,NaN,2016-04-17T16:15:32,"{'user_mentions': [], 'symbols': [], 'urls': [...",NaN,{'@text': '“Muchos neoyorquinos luchan por pag...,<__main__.DbpediaObject object at 0x7f0b88d9ba90>,0.000000,0.000000


In [28]:
polarity_mean = []
polarity_median = []
for index, el in entities_frame.iterrows():
  partial = list()
  for i in el['occurrences']:
    partial.append(clinton.iloc[i]['sentiment'])
  polarity_mean.append(statistics.mean(partial))
  polarity_median.append(statistics.median(partial))


In [29]:
entities_frame['sentiment_mean'] = polarity_mean
entities_frame['sentiment_median'] = polarity_median

In [30]:
entities_frame.sort_values('count', ascending=False)

,entita,label,count,types,occurrences,plain_text,sentiment_mean,sentiment_median
1,<__main__.Entity object at 0x7f0b88df6da0>,Donald_Trump,884,"[Person, Agent, Politician]","{2048, 1, 2050, 2051, 6, 7, 2055, 9, 10, 2056,...","{Donald Trump, Trump, President Trump}",0.064329,0.000000
2,<__main__.Entity object at 0x7f0b88e07f60>,President_of_the_United_States,396,[],"{2, 10, 16, 25, 2081, 2083, 2085, 2087, 2088, ...","{President, @POTUS, United States presidency, ...",0.143895,0.000000
8,<__main__.Entity object at 0x7f0b88e0e710>,United_States,336,"[PopulatedPlace, Place, Location, Country]","{2049, 1026, 1027, 1542, 9, 2057, 1551, 2063, ...","{U.S., USA, Estados Unidos, americanos, US, Un...",0.143811,0.068182
52,<__main__.Entity object at 0x7f0b88e34048>,Donald_Trump_on_social_media,116,[],"{2057, 2573, 2062, 1042, 2067, 2578, 2069, 258...","{@realdonaldtrump, @realDonaldTrump, @RealDona...",0.044192,0.000000
44,<__main__.Entity object at 0x7f0b88d9b908>,Hillary_Clinton,108,"[Person, Agent, Politician]","{1536, 1540, 1542, 2570, 1548, 1559, 1560, 156...","{Hillary Clinton, Clinton}",0.155497,0.007407
...,...,...,...,...,...,...,...,...
906,<__main__.Entity object at 0x7f0b891a5550>,Iron,1,[],{1402},{fe},0.000000,0.000000
269,<__main__.Entity object at 0x7f0b893eb6a0>,Symposium,1,[],{340},{Symposium},-0.015152,-0.015152
270,<__main__.Entity object at 0x7f0b893eb978>,Weather,1,[],{343},{weather},-0.200000,-0.200000
903,<__main__.Entity object at 0x7f0b891aabe0>,Trust_law,1,[],{1399},{trusts},-0.187500,-0.187500


In [33]:
entities_frame[entities_frame['label']=='Russia']

,entita,label,count,types,occurrences,plain_text,sentiment_mean,sentiment_median
60,<__main__.Entity object at 0x7f0b890ded68>,Russia,8,"[PopulatedPlace, Place, Location, Country]","{65, 66, 1090, 165, 933, 1105, 1746, 568}",{Russia},0.023958,-0.1


In [ ]:
# entities_frame.to_csv('entities_clinton.csv')

# Tabella entità usate in comune

In [ ]:
# partial_matrix = np.zeros([len(entities_frame), len(entities_frame)], dtype=int)
# for i in range(0,len(entities_frame)):
  # print(i, end= ' ')
  # for j in range(i+1,len(entities_frame)):
    # partial_matrix[i,j] = len(entities_frame.iloc[i]['occurrences'].intersection(entities_frame.iloc[j]['occurrences']))

In [ ]:
# np.savetxt("clinton_entities_matrix.csv", partial_matrix, delimiter=",")

In [36]:
clinton_entities_matrix = np.genfromtxt('/content/drive/MyDrive/ProgettoAI/dataset/clinton_entities_matrix.csv', delimiter=',')

#Codice specifico

In [ ]:
# per utilizzarla inserire all'interno della query il token "TEXT_HERE" dove si vuole inserire l'uri dell'entità
def party_from_uri(uri_list, query, variable):
  tot = []
  for uri in uri_list:
    # print(uri)
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query.replace('TEXT_HERE', uri))   
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    # tot.append(results["results"]["bindings"])
    if len(results["results"]["bindings"]) != 0:
      party = results["results"]["bindings"][0][variable]
      res_entity = {
              "res" :uri,
              variable: party["value"],
          }
    else:
      res_entity = {
              "res" :uri,
              variable: None,
          }
    tot.append(res_entity)
  return tot



#politici piu nominati

In [ ]:
politici = entita_filter_by_type('Politician', entities_frame)
query ="""
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
              SELECT ?party_name
              WHERE {  
                 <TEXT_HERE>   dbp:party ?party_uri.
                 ?party_uri dbp:name ?party_name              
               }
          """
partiti = party_from_uri(entita_filter_by_type('Politician',entities_frame)['uri'], query, 'party_name')

In [ ]:
politici['party'] = [x['party_name'] for x in partiti]
print('Republican Party')
display(politici[politici['party']=='Republican Party'].iloc[0:6])
print('--------------------------------------------------------------')
print('Democratic Party')
display(politici[politici['party']=='Democratic Party'].iloc[0:6])

Republican Party


,label,count,uri,plain_text,party
0,Donald_Trump,884,http://dbpedia.org/resource/Donald_Trump,"{President Trump, Trump, Donald Trump}",Republican Party
16,Mike_Pence,26,http://dbpedia.org/resource/Mike_Pence,"{VP, Mike Pence}",Republican Party
20,Ted_Cruz,6,http://dbpedia.org/resource/Ted_Cruz,{Ted Cruz},Republican Party
24,Abraham_Lincoln,6,http://dbpedia.org/resource/Abraham_Lincoln,{Lincoln},Republican Party
21,David_Duke,5,http://dbpedia.org/resource/David_Duke,{David Duke},Republican Party
17,John_McCain,4,http://dbpedia.org/resource/John_McCain,"{McCain, John McCain}",Republican Party


--------------------------------------------------------------
Democratic Party


,label,count,uri,plain_text,party
1,Hillary_Clinton,108,http://dbpedia.org/resource/Hillary_Clinton,"{Hillary Clinton, Clinton}",Democratic Party
2,Barack_Obama,40,http://dbpedia.org/resource/Barack_Obama,"{Barack Obama, President Obama, Obama}",Democratic Party
4,Bill_Clinton,13,http://dbpedia.org/resource/Bill_Clinton,"{Clinton, Bill Clinton}",Democratic Party
29,Bernie_Sanders,6,http://dbpedia.org/resource/Bernie_Sanders,"{Bernie Sanders, Sanders}",Democratic Party
5,Tim_Kaine,6,http://dbpedia.org/resource/Tim_Kaine,"{Tim Kaine, Kaine}",Democratic Party
9,Franklin_D._Roosevelt,5,http://dbpedia.org/resource/Franklin_D._Roosevelt,"{Franklin Roosevelt, FDR}",Democratic Party


#stati americani + citati

In [ ]:
entita_filter_by_type('Location', entities_frame)

query ="""
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
              SELECT ?name
              WHERE {  
                 <TEXT_HERE>   dbo:country ?country.
                 ?country dbp:conventionalLongName ?name.
               }
          """
countries = party_from_uri(entita_filter_by_type('Location',entities_frame)['uri'], query, 'name')

In [ ]:
luoghi_usa = entita_filter_by_type('Location',entities_frame)
luoghi_usa['country']= [x['name'] for x in countries]
luoghi_usa[luoghi_usa['country'] == 'United States of America']

,label,count,uri,plain_text,country
8,New_York_City,28,http://dbpedia.org/resource/New_York_City,"{New York, Nueva York, NY, NYC}",United States of America
9,Virginia,13,http://dbpedia.org/resource/Virginia,"{va, VA, Virginia}",United States of America
15,Indiana,11,http://dbpedia.org/resource/Indiana,"{Indiana, IN}",United States of America
56,Pennsylvania,11,http://dbpedia.org/resource/Pennsylvania,"{Pennsylvania, PA}",United States of America
18,Ohio,10,http://dbpedia.org/resource/Ohio,{Ohio},United States of America
1,North_Carolina,10,http://dbpedia.org/resource/North_Carolina,"{NC, North, North Carolina}",United States of America
26,California,9,http://dbpedia.org/resource/California,"{California, CA}",United States of America
35,Texas,8,http://dbpedia.org/resource/Texas,"{Texas, TX}",United States of America
47,Connecticut,8,http://dbpedia.org/resource/Connecticut,"{CT, Connecticut}",United States of America
28,Philadelphia,8,http://dbpedia.org/resource/Philadelphia,{Philadelphia},United States of America


# persone non politiche citate


In [ ]:
pd.concat([entita_filter_by_type('Person', entities_frame), politici]).drop_duplicates(subset=['label'], keep = False).iloc[0:10]


,label,count,uri,plain_text,party
1,Gabby_Barrett,4,http://dbpedia.org/resource/Gabby_Barrett,{I hope},NaN
99,Aidan_of_Lindisfarne,3,http://dbpedia.org/resource/Aidan_of_Lindisfarne,{Aidan},NaN
2,Alicia_Machado,3,http://dbpedia.org/resource/Alicia_Machado,{Alicia Machado},NaN
89,Elie_Wiesel,2,http://dbpedia.org/resource/Elie_Wiesel,{Elie Wiesel},NaN
86,Mark_Ella,2,http://dbpedia.org/resource/Mark_Ella,{ella},NaN
101,Sally_Ride,2,http://dbpedia.org/resource/Sally_Ride,{Sally Ride},NaN
67,Madeleine_of_Valois,2,http://dbpedia.org/resource/Madeleine_of_Valois,{Madeleine},NaN
21,Ella_Chen,2,http://dbpedia.org/resource/Ella_Chen,"{ella, Ella}",NaN
15,Rosa_Parks,2,http://dbpedia.org/resource/Rosa_Parks,{Rosa Parks},NaN
27,Sido_(rapper),2,http://dbpedia.org/resource/Sido_(rapper),{sido},NaN


#schieramento sugli stati (ecluso USA)


In [ ]:
luoghi = entita_filter_by_type('Country', entities_frame)
luoghi.iloc[0:7]

,label,count,uri,plain_text
0,United_States,336,http://dbpedia.org/resource/United_States,"{United States, americanos, U.S., American, US..."
5,Mexico,18,http://dbpedia.org/resource/Mexico,"{Mexico, Mexicans, Mexican}"
3,Russia,8,http://dbpedia.org/resource/Russia,{Russia}
2,Iraq,7,http://dbpedia.org/resource/Iraq,{Iraq}
12,Bangladesh,5,http://dbpedia.org/resource/Bangladesh,{Bangladesh}
9,Iran,4,http://dbpedia.org/resource/Iran,"{Iranian, Iran}"
13,China,4,http://dbpedia.org/resource/China,{China}


In [46]:
print(clinton_entities_matrix[271,:].argmax())
print(clinton_entities_matrix[:,271].argmax())
# entities_frame.iloc[535]

478
1


In [49]:
clinton_entities_matrix[271,478]

7.0

In [50]:
entities_frame.iloc[478]

entita                     <__main__.Entity object at 0x7f0b88e07358>
label                                                     Deportation
count                                                              14
types                                                              []
occurrences         {2944, 678, 647, 712, 3208, 3211, 2957, 785, 6...
plain_text                    {deport, deportation, mass deportation}
sentiment_mean                                              0.0204296
sentiment_median                                                    0
Name: 478, dtype: object

In [ ]:
clinton.iloc[2060]['text']

'Praying for a safe Eid Al-Fitr. My heart breaks for families struck by terror in Turkey, Iraq, Saudi Arabia, and Bangladesh this Ramadan. -H'

In [44]:
entities_frame[entities_frame['label']=='Immigration']

,entita,label,count,types,occurrences,plain_text,sentiment_mean,sentiment_median
271,<__main__.Entity object at 0x7f0b88e07b00>,Immigration,39,[],"{2944, 641, 3205, 647, 652, 653, 2957, 785, 65...","{immigrants, immigration, immigrant}",-0.039906,0.0


In [ ]:
clinton.iloc[419]['text']

'5. How can you be tough on Iran, given your business partnership with someone connected to Iranian money laundering? https://t.co/1AdDm2kfvy'

# Organizzazioni

In [ ]:
# Organisation 
# PoliticalParty 
# Company 
# Legislature
# Broadcaster
# PeriodicalLiterature 

In [ ]:
pd.concat([entita_filter_by_type('Organisation', entities_frame), entita_filter_by_type('PoliticalParty', entities_frame)]).drop_duplicates(subset=['label'], keep = False).iloc[0:10]
# tutti le Organizzazioni che non siano un partito politico

,label,count,uri,plain_text
17,United_States_Senate,29,http://dbpedia.org/resource/United_States_Senate,"{Senator, United States Senate, senator, Senate}"
2,RT_(TV_network),26,http://dbpedia.org/resource/RT_(TV_network),{RT}
9,Orlando_Magic,18,http://dbpedia.org/resource/Orlando_Magic,{Orlando}
26,Portland_Timbers_U23s,17,http://dbpedia.org/resource/Portland_Timbers_U23s,{por}
21,Trump_University,17,http://dbpedia.org/resource/Trump_University,{Trump University}
3,Imagine_Software,16,http://dbpedia.org/resource/Imagine_Software,{Imagine}
45,United_States_Congress,15,http://dbpedia.org/resource/United_States_Cong...,{Congress}
18,United_States_House_of_Representatives,9,http://dbpedia.org/resource/United_States_Hous...,"{House, H.R.}"
80,Planned_Parenthood,7,http://dbpedia.org/resource/Planned_Parenthood,{Planned Parenthood}
51,IF_Ready,7,http://dbpedia.org/resource/IF_Ready,{Ready}


In [ ]:
entita_filter_by_type('Legislature', entities_frame)

,label,count,uri,plain_text
0,United_States_Senate,29,http://dbpedia.org/resource/United_States_Senate,"{Senator, United States Senate, senator, Senate}"
2,United_States_Congress,15,http://dbpedia.org/resource/United_States_Cong...,{Congress}
1,United_States_House_of_Representatives,9,http://dbpedia.org/resource/United_States_Hous...,"{House, H.R.}"
3,Senate_of_Spain,1,http://dbpedia.org/resource/Senate_of_Spain,{Senado}


In [ ]:
entita_filter_by_type('Broadcaster', entities_frame).iloc[0:5]

,label,count,uri,plain_text
0,RT_(TV_network),26,http://dbpedia.org/resource/RT_(TV_network),{RT}
2,KTSU,7,http://dbpedia.org/resource/KTSU,{The choice}
9,CNN,3,http://dbpedia.org/resource/CNN,{CNN}
3,MSNBC,2,http://dbpedia.org/resource/MSNBC,{MSNBC}
1,NEW_(TV_station),1,http://dbpedia.org/resource/NEW_(TV_station),{NEW}


In [ ]:
entita_filter_by_type('PeriodicalLiterature', entities_frame).iloc[0:5]

,label,count,uri,plain_text
0,The_New_York_Times,11,http://dbpedia.org/resource/The_New_York_Times,"{NYTimes, NYT, nytimes}"
2,Respect_(magazine),6,http://dbpedia.org/resource/Respect_(magazine),{respect.}
4,Newsweek,3,http://dbpedia.org/resource/Newsweek,{Newsweek}
7,Look_(American_magazine),2,http://dbpedia.org/resource/Look_(American_mag...,{Look}
8,Fact_(UK_magazine),2,http://dbpedia.org/resource/Fact_(UK_magazine),{FACT}


#Sotituzione entità nel testo

In [ ]:
final_text = []
for index, row in clinton.iterrows():
  partial = row['text']
  for ent in row['dbpedia_object'].entities:
    partial=partial.replace(ent.original_text, re.sub('[^A-Za-z0-9]+', '', ent.label))
  final_text.append(partial)


In [ ]:
pd.DataFrame(final_text, columns=['tweet']).to_csv('tweet_with_entities_clinton.csv')